# Подготовка данных для обучения моделей

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* https://scikit-learn.org/stable/modules/compose.html#pipeline-chaining-estimators
* https://pytorch.org/docs/stable/data.html
* https://pytorch.org/tutorials/beginner/data_loading_tutorial.html
* Deep Learning with PyTorch (2020) Авторы: Eli Stevens, Luca Antiga, Thomas Viehmann


## Задачи для совместного разбора

1. Рассмотрите, как можно выстраивать конвейер предобработки данных при помощи `Pipeline` из `sklearn`

In [ ]:
from sklearn.datasets import make_regression
from sklearn.preprocessing import MinMaxScaler, StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline

In [ ]:
X, y = make_regression(
    n_samples=1000,
    n_features=5
)

In [ ]:
pipe = Pipeline(
    [
        ("scaling", MinMaxScaler()),
        ("poly", PolynomialFeatures()),
        ("lr", LinearRegression()),
    ]
).fit(X, y)

In [ ]:
pipe.predict(X)

array([-8.34780553e+01,  9.54880148e+01, -9.68378773e+00,  2.46529733e+02,
       -1.55189964e+01,  1.51737314e+02, -9.31799219e+01,  5.44227897e+01,
        1.09621479e+02, -1.03513200e+02, -4.93189400e+01, -9.19910961e+01,
        5.23922193e+01,  3.02704791e+01,  1.18377665e+02,  1.50194774e+02,
       -4.83683298e+01, -1.56438610e+02,  3.46744862e+00, -9.02750898e+01,
        2.57937978e+02,  5.59211967e+01,  3.22894667e+01, -1.05889493e+02,
       -1.99986075e+02,  1.86278322e+02, -7.23986664e+01, -3.57070373e+01,
        7.67176881e+01, -1.06523891e+02,  9.97967723e+01,  7.98905884e+01,
       -1.14684067e+00, -9.67814643e+01,  6.03303171e+01,  4.62233148e+01,
        4.30302806e+01,  1.71315240e+02, -4.03277745e+01, -8.81202088e+01,
        4.54076953e+00,  1.48805238e+02, -4.73326922e+01,  5.92731591e+01,
       -8.80719757e+00,  8.44212564e+01, -2.02073454e+02, -1.27786494e+02,
        7.60540902e+01,  5.92836857e+01, -1.51818634e+02,  1.25864163e+02,
        6.84660350e+01,  

2. Создайте синтетический датасет для задачи регрессии и представьте его в виде `torch.utils.data.Dataset`

In [ ]:
from torch.utils.data import Dataset, DataLoader

In [ ]:
a = [1, 2, 3]
a[2] # a.__getitem__(2)
len(a)

3

In [ ]:
class SyntDataset(Dataset):
  def __init__(self, transform: callable = None, **make_regression_args):
    self.X, self.y = make_regression(**make_regression_args)
    # self.X = th.Tensor(self.X)
    self.transform = transform

  def __getitem__(self, idx):
    # return {
    #     "x": self.X[idx],
    #     "y": self.y[idx]
    # }
    x = self.X[idx]
    if self.transform is not None:
      x = self.transform(x)
    return x, self.y[idx]

  def __len__(self):
    return len(self.X)

In [ ]:
import numpy as np
def add_squares(x):
  return np.c_[x, x ** 2]

In [ ]:
x = np.random.randint(0, 200, (20, 5))
x.shape

(20, 5)

In [ ]:
add_squares(x).shape

(20, 10)

In [ ]:
class SquareN:
  def __init__(self, n):
    self.n = n

  def __call__(self, x):
    for _ in range(self.n):
      x = add_squares(x)
    return x

In [ ]:
o = SquareN(2)
o
# o(d.X[:4])
# add_squares(d.X[:4])

In [ ]:
# d = SyntDataset(n_samples=1000, n_features=5)
d = SyntDataset(
    transform=SquareN(2),
    n_samples=1000,
    n_features=5
)
d[0:5]

(array([[-1.02869994e+00, -3.67097166e-01, -4.30240336e-01,
          2.11961902e-01, -1.24492671e-01,  1.05822356e+00,
          1.34760329e-01,  1.85106747e-01,  4.49278480e-02,
          1.54984252e-02,  1.05822356e+00,  1.34760329e-01,
          1.85106747e-01,  4.49278480e-02,  1.54984252e-02,
          1.11983710e+00,  1.81603464e-02,  3.42645077e-02,
          2.01851153e-03,  2.40201184e-04],
        [ 3.11243690e-02,  9.49279608e-01,  1.23526083e+00,
         -4.47978555e-02, -1.94604707e-01,  9.68726346e-04,
          9.01131774e-01,  1.52586931e+00,  2.00684786e-03,
          3.78709918e-02,  9.68726346e-04,  9.01131774e-01,
          1.52586931e+00,  2.00684786e-03,  3.78709918e-02,
          9.38430733e-07,  8.12038473e-01,  2.32827717e+00,
          4.02743832e-06,  1.43421202e-03],
        [ 1.78453670e-01, -8.47716350e-02, -1.40162526e+00,
          7.83158292e-01,  5.70984722e-02,  3.18457122e-02,
          7.18623010e-03,  1.96455338e+00,  6.13336910e-01,
          3.

In [ ]:
d[:5]

(array([[-1.02869994e+00, -3.67097166e-01, -4.30240336e-01,
          2.11961902e-01, -1.24492671e-01,  1.05822356e+00,
          1.34760329e-01,  1.85106747e-01,  4.49278480e-02,
          1.54984252e-02,  1.05822356e+00,  1.34760329e-01,
          1.85106747e-01,  4.49278480e-02,  1.54984252e-02,
          1.11983710e+00,  1.81603464e-02,  3.42645077e-02,
          2.01851153e-03,  2.40201184e-04],
        [ 3.11243690e-02,  9.49279608e-01,  1.23526083e+00,
         -4.47978555e-02, -1.94604707e-01,  9.68726346e-04,
          9.01131774e-01,  1.52586931e+00,  2.00684786e-03,
          3.78709918e-02,  9.68726346e-04,  9.01131774e-01,
          1.52586931e+00,  2.00684786e-03,  3.78709918e-02,
          9.38430733e-07,  8.12038473e-01,  2.32827717e+00,
          4.02743832e-06,  1.43421202e-03],
        [ 1.78453670e-01, -8.47716350e-02, -1.40162526e+00,
          7.83158292e-01,  5.70984722e-02,  3.18457122e-02,
          7.18623010e-03,  1.96455338e+00,  6.13336910e-01,
          3.

In [ ]:
dl = DataLoader(d, batch_size=32)

In [ ]:
for (batch_X, batch_y) in dl:
  print(batch_X.shape, batch_y.shape)
  break

torch.Size([32, 5, 4]) torch.Size([32])


In [ ]:
def clip(x):
  return np.clip(x, 0, 1)

def add_squaresN(x, n):
  for _ in range(n):
    x = add_squares(x)
  return x

In [ ]:
x = np.random.randint(0, 200, (20, 5))
# x = clip(x)
# x = add_squaresN(x, 2)

for f in [clip, SquareN(2)]:
  x = f(x)

## Задачи для самостоятельного решения

<p class="task" id="1"></p>

1\. Считайте файл `bank-full.csv` ([источник](https://www.kaggle.com/datasets/hariharanpavan/bank-marketing-dataset-analysis-classification)) в виде `pd.DataFrame`. Используя `Pipeline` из `sklearn`, закодируйте значения в нечисловых столбцах целыми числами, после чего нормализуйте получившиеся признаки. Выведите преобразованные данные на экран.

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('bank-full.csv')
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [ ]:
X = df.drop(columns = ['y'])
y  = df['y']

In [ ]:
pip install category_encoders

In [ ]:
import category_encoders as ce

pipe = Pipeline(
    steps = [
        ('encoder', ce.OrdinalEncoder()),
        ('scaling', StandardScaler())
    ]
).fit(X,y)

In [ ]:
pd.DataFrame(pipe.transform(X)).head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,1.606965,-1.138083,-0.740666,-1.083560,-0.13549,0.256419,-0.893915,-0.436803,-1.412108,-1.298476,-0.924841,0.011016,-0.569351,-0.411453,-0.25194,-0.415287
1,0.288529,-0.815326,0.702458,-0.051543,-0.13549,-0.437895,-0.893915,-0.436803,-1.412108,-1.298476,-0.924841,-0.416127,-0.569351,-0.411453,-0.25194,-0.415287
2,-0.747384,-0.492570,-0.740666,-0.051543,-0.13549,-0.446762,-0.893915,2.289359,-1.412108,-1.298476,-0.924841,-0.707361,-0.569351,-0.411453,-0.25194,-0.415287
3,0.571051,-0.169813,-0.740666,0.980475,-0.13549,0.047205,-0.893915,-0.436803,-1.412108,-1.298476,-0.924841,-0.645231,-0.569351,-0.411453,-0.25194,-0.415287
4,-0.747384,0.152944,0.702458,0.980475,-0.13549,-0.447091,1.118674,-0.436803,-1.412108,-1.298476,-0.924841,-0.233620,-0.569351,-0.411453,-0.25194,-0.415287


<p class="task" id="2"></p>

2\. Опишите класс `BankDatasetBase`. Решение должно удовлетворять следующим критериям:

* класс наследуется от `torch.utils.data.Dataset`;
* при создании объекта в конструктор передается набор данных в виде `pd.DataFrame`;
* объекты класса имеют поля `X` и `y` с признаками и метками соответственно;
* класс реализует интерфейс последовательностей (`__getitem__` + `__len__`);
* `obj[i]` возвращает кортеж, содержащий `i`-ую строку из `obj.X` и `i`-ую строку из `obj.y`.
    
Создайте объект класса `BankDatasetBase` и продемонстрируйте работоспособность.

In [ ]:
from torch.utils.data import Dataset
import pandas as pd

class BankDatasetBase(
    Dataset
):
    def __init__(self, data: pd.DataFrame) -> None:
        self.X = data.drop(columns = ['y'])
        self.y = data['y']

    def __getitem__(self, idx) -> tuple:
        return (self.X.iloc[idx], self.y.iloc[idx])

    def __len__(self) -> int:
      return self.X.shape[0]
        # pass

In [ ]:
data = pd.read_csv('/content/bank-full.csv')
data.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [ ]:
obj = BankDatasetBase(data)
obj[0]

(age                  58
 job          management
 marital         married
 education      tertiary
 default              no
 balance            2143
 housing             yes
 loan                 no
 contact         unknown
 day                   5
 month               may
 duration            261
 campaign              1
 pdays                -1
 previous              0
 poutcome        unknown
 Name: 0, dtype: object,
 'no')

In [ ]:
len(obj)

45211

<p class="task" id="3"></p>

3\. Опишите класс `BankDataset`. Решение должно удовлетворять всем критериям из предыдущего задания, а также:
* при создании объекта в конструктор может быть передан необязательный аргумент `transform: callable`;
* если данный аргумент был передан, то при получении `i`-го элемента, нужно вызвать `transform(x, y)` и вернуть полученный результат.

Создайте объект класса `BankDataset` и продемонстрируйте работоспособность.

In [ ]:
class BankDataset(
    Dataset
):
    def __init__(self, data: pd.DataFrame, transform: callable = None) -> None:
        # pass
        self.X = pd.DataFrame(data.drop(['y'], axis = 1))
        self.y = pd.DataFrame(data['y'])
        self.transform = transform


    def __getitem__(self, idx) -> tuple:
        # x - набор признаков из idx-й строки
        # y - набор признаков из idx-й строки
        # если при создании был передан transform
        # X, y = transform(X, y)

        x, y = self.X.iloc[idx], self.y.iloc[idx]
        if self.transform is not None:
          x, y = self.transform(x, y)
        return(x,y)

    def __len__(self) -> int:
        return self.X.shape[0]

In [ ]:
def transform_func(x_i, y_i):
  x_i *= 5
  y_i = y_i.replace("no", '0')
  y_i = y_i.replace("yes", '1')
  return (x_i, y_i)

In [ ]:
data = pd.read_csv('/content/bank-full.csv')
data.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [ ]:
data.shape

(45211, 17)

In [ ]:
print(*BankDataset(data, transform_func)[2])

age                                                        165
job          entrepreneurentrepreneurentrepreneurentreprene...
marital                    marriedmarriedmarriedmarriedmarried
education        secondarysecondarysecondarysecondarysecondary
default                                             nonononono
balance                                                     10
housing                                        yesyesyesyesyes
loan                                           yesyesyesyesyes
contact                    unknownunknownunknownunknownunknown
day                                                         25
month                                          maymaymaymaymay
duration                                                   380
campaign                                                     5
pdays                                                       -5
previous                                                     0
poutcome                   unknownunknownunknownunknown

In [ ]:
len(BankDataset(data, transform_func))

45211

<p class="task" id="4"></p>

4\. Опишите класс `LabelEncoderTransform`. Решение должно удовлетворять следующим критериям:

* при создании объекта в конструктор передаются названия нечисловых столбцов в датасете
* класс реализует интерфейс `Callable` (`__call__`); метод `__call__` имеет два параметра (признаки и класс) и возвращает кортеж, состоящих двух элементов: набора признаков, в котором нечисловые характеристики закодированы целыми числами; и чисел - меток классов;
* состояние объекта (индексы для кодирования) обновляется в момент очередного вызова `__call__` (т.е. все данные сразу никогда не передаются никакому методу объекта).
    
Продемонстрируйте работоспособность, создав объект `BankDataset` и передав при создании объект класса `LabelEncoderTransform`.

In [ ]:
x = pd.DataFrame(BankDataset(data)[3:50][0])
y = pd.DataFrame(BankDataset(data)[3:50][1])
y.head()

,y
3,no
4,no
5,no
6,no
7,no


In [ ]:
# perem = y[y.columns[0]].unique()   # все уникальные знчаения столбца y
# print(perem)
# map = {perem[i]:i for i in range(len(perem))}
# print(map)
# np.array(y.apply(lambda x: x.map(map))).ravel()

In [ ]:
from sklearn.preprocessing import LabelEncoder

class LabelEncoderTransform:
    def __init__(self, category_columns: list[str]) -> None:
        self.category_columns = category_columns

    def __call__(self, x, y) -> tuple:
        x_le = []
        y_le = []
        cat_cols_x = []   # названия категориальных столбцов для Х

        for column in self.category_columns:
          if y.columns == column:   # если y -- категориальная переменная
            # y_le = LabelEncoder().fit_transform(y)
            perem = y[y.columns[0]].unique()   # все уникальные значения столбца y
            map = {perem[i]:i for i in range(len(perem))}
            y_le = np.array(y.apply(lambda x: x.map(map))).ravel()

          else:
            cat_cols_x.append(column)
            y_le = y  # если y колич перем, то остается без изменений

        # cat_cols_x
        x_cat = x[cat_cols_x]   # значения только категориальных признаков X
        x_le = ce.OrdinalEncoder().fit_transform(x_cat)


        x_num = [i for i in x.columns if i not in cat_col]   # добавление кол признаков
        x_res = np.hstack((x[x_num], x_le))

        return (x_res, y_le)

In [ ]:
LabelEncoderTransform(["qwe"])

In [ ]:
import pandas as pd
data = pd.read_csv('bank-full.csv')
data.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [ ]:
num_col = data._get_numeric_data().columns
num_col

Index(['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous'], dtype='object')

In [ ]:
cat_col = list(data.drop(columns = num_col).columns)
cat_col

['job',
 'marital',
 'education',
 'default',
 'housing',
 'loan',
 'contact',
 'month',
 'poutcome',
 'y']

In [ ]:
# num_col = [i for i in data.columns if i not in cat_col]
# num_col

In [ ]:
encoder = LabelEncoderTransform(cat_col)

In [ ]:
BankDataset(data, encoder)[8:12]

(array([[ 58, 121,   5,  50,   1,  -1,   0,   1,   1,   1,   1,   1,   1,
           1,   1,   1],
        [ 43, 593,   5,  55,   1,  -1,   0,   2,   2,   2,   1,   1,   1,
           1,   1,   1],
        [ 41, 270,   5, 222,   1,  -1,   0,   3,   3,   2,   1,   1,   1,
           1,   1,   1],
        [ 29, 390,   5, 137,   1,  -1,   0,   3,   2,   2,   1,   1,   1,
           1,   1,   1]]),
 array([0, 0, 0, 0]))

<p class="task" id="5"></p>

5\. Опишите класс `ToTensor`.  Решение должно удовлетворять следующим критериям:
* класс реализует интерфейс `Callable` (`__call__`); метод `__call__` принимает имеет параметра (признаки и класс в числовом виде) и возвращает кортеж, состоящих 2 тензоров.

Опишите класс `Compose`.  Решение должно удовлетворять следующим критериям:
* при создании объекта в конструктор передается список объектов `transforms`, каждый из которых имеет метод `__call__(x, y)`;
* класс реализует интерфейс `Callable` (`__call__`); метод `__call__` принимает имеет параметра (признаки и класс в числовом виде) и и возвращает кортеж, полученный путем последовательного вызова объектов из `transforms`.

Продемонстрируйте работоспособность, создав объект `BankDataset` и передав при создании список из объектов LabelEncoderTransform и ToTensor.

In [ ]:
import torch as th

class ToTensor(object):
    def __call__(self, X, y) -> tuple:
        return (th.Tensor(X).long(), th.Tensor(y).long())

class Compose(object):
    def __init__(self, transforms: list) -> None:
        self.transforms = transforms  # список функций/методов

    def __call__(self, X, y):
        for trans_func in self.transforms:
          X,y = trans_func(X,y)
        return X, y

In [ ]:
transf = Compose([LabelEncoderTransform(cat_col), ToTensor()])

In [ ]:
BankDataset(data, transf)[8:12]

(tensor([[ 58, 121,   5,  50,   1,  -1,   0,   1,   1,   1,   1,   1,   1,   1,
            1,   1],
         [ 43, 593,   5,  55,   1,  -1,   0,   2,   2,   2,   1,   1,   1,   1,
            1,   1],
         [ 41, 270,   5, 222,   1,  -1,   0,   3,   3,   2,   1,   1,   1,   1,
            1,   1],
         [ 29, 390,   5, 137,   1,  -1,   0,   3,   2,   2,   1,   1,   1,   1,
            1,   1]]),
 tensor([0, 0, 0, 0]))

In [ ]:
len(BankDataset(data, encoder))

45211

<p class="task" id="6"></p>

6\. Разделите датасет из предыдущего задания на обучающую и тестовую выборку в соотношении 75% на 25%. Создайте объект `DataLoader` для получения пакетов размера 64, полученных из перемешанного обучающего датасета. Кастомизируйте `DataLoader` таким образом, чтобы пакет признаков был представлен в виде трехмерного тензора размера 64x2x8 (разделите 16 признаков на два тензора по 8). Получите один пакет и выведите на экран размерность тензоров пакета.

> Upd:\
Сначала разбить на трэйн/ тест, потом к трэйну применить ```BankDataset(data, transform = transf)[0:len(data)]``` и не делать ```TensorDataset```

In [ ]:
transf = Compose([LabelEncoderTransform(cat_col), ToTensor()])

In [ ]:
# преобразование категориальных признаков

data_transf = BankDataset(data, transform = transf)[0:len(data)]
data_transf

(tensor([[  58, 2143,    5,  ...,    1,    1,    1],
         [  44,   29,    5,  ...,    1,    1,    1],
         [  33,    2,    5,  ...,    1,    1,    1],
         ...,
         [  72, 5715,   17,  ...,    2,    6,    4],
         [  57,  668,   17,  ...,    3,    6,    1],
         [  37, 2971,   17,  ...,    2,    6,    3]]),
 tensor([0, 0, 0,  ..., 1, 0, 0]))

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    data_transf[0], data_transf[1],
    test_size = 0.25,
    random_state = 42
)
X_train, X_train.shape

(tensor([[  52,  118,   25,  ...,    2,   12,    1],
         [  28,  459,   16,  ...,    2,   10,    1],
         [  36,  156,   22,  ...,    2,    4,    1],
         ...,
         [  34, 1317,   15,  ...,    2,    1,    1],
         [  33,  165,    7,  ...,    1,    1,    1],
         [  38,  -41,   21,  ...,    2,    3,    1]]),
 torch.Size([33908, 16]))

In [ ]:
from torch.utils.data import TensorDataset
dataset_train = TensorDataset(X_train, y_train)
# len(dataset_train[0:][0])

In [ ]:
data_loader = DataLoader(
    dataset = dataset_train,
    batch_size= 64,    # кол-во образцов в пакете для загрузки
    shuffle = True, # перетасовка
)

In [ ]:
paket = next(iter(data_loader))
paket   # получение 1 пакета

[tensor([[  35, 5260,   20,  ...,    1,    1,    1],
         [  55, 8304,    1,  ...,    2,   12,    4],
         [  37,  281,   16,  ...,    1,    1,    1],
         ...,
         [  35, 7530,   18,  ...,    2,    6,    1],
         [  47,  186,    2,  ...,    2,    9,    1],
         [  40, 1108,   11,  ...,    2,    4,    4]]),
 tensor([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
         0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])]

In [ ]:
X_iter, y_iter = paket

In [ ]:
X_iter = X_iter.reshape((64,2,8))
X_iter

tensor([[[  35, 5260,   20,  ...,   -1,    0,    1],
         [   2,    2,    1,  ...,    1,    1,    1]],

        [[  55, 8304,    1,  ...,   92,    1,    7],
         [   1,    2,    1,  ...,    2,   12,    4]],

        [[  37,  281,   16,  ...,   -1,    0,    7],
         [   1,    2,    1,  ...,    1,    1,    1]],

        ...,

        [[  35, 7530,   18,  ...,   -1,    0,    2],
         [   2,    2,    1,  ...,    2,    6,    1]],

        [[  47,  186,    2,  ...,   -1,    0,    3],
         [   3,    2,    1,  ...,    2,    9,    1]],

        [[  40, 1108,   11,  ...,  112,    1,    7],
         [   1,    1,    1,  ...,    2,    4,    4]]])

In [ ]:
X_iter.size(), y_iter.size()

(torch.Size([64, 2, 8]), torch.Size([64]))

## Обратная связь
- [x] Хочу получить обратную связь по решению